In [1]:
import re
import requests
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup as soup
from typing import List, Union, Dict

In [2]:
BASE_URL_DST_FINAL = 'https://wdc.kugi.kyoto-u.ac.jp/dst_final/'

In [46]:
def get_month_data(year: str, month: str) -> Union[str, Exception]:
    """
    year [str]: YYYYY
    month [str]: DD
    """
    url = f"{BASE_URL_DST_FINAL}/{year}{month}/index.html"
    try:
        ans = requests.get(url=url)
        data = soup(ans.text, "html.parser")
        text_from_html = data.findAll("pre")[0].text
        return text_from_html
    except Exception as error:
        raise error

In [47]:
def clean_month_data_text(data_text: str) -> List[str]:
    """
    data_text [str]
    """
    raw_data = list()
    _ = [raw_data.append(i) for i in data_text.split('\n') if i != '']
    return raw_data[6:]


def clean_single_line(line: str) -> List[int]:
    """
    line [str]
    """
    # seleciona somente os valores de dst dentro da lista
    # quebra o texto em 3 blocos com 33 caracteres
    split_three_blocks = re.findall('.................................', line[2:])
    # remove o primeiro caracter de cada bloco
    clean_blocks = list()
    _ = [clean_blocks.append(i[1:]) for i in split_three_blocks]
    # separa os blocos em conjuntos de 4 caracteres
    # converte valores de string para inteiro
    separated_values = [int(i) for i in re.findall('....', ''.join(clean_blocks))]
    return separated_values


def clean_multiples_lines(lines: List[str]):
    lines_ok = list()
    _ = [lines_ok.append(clean_single_line(i)) for i in lines]
    return lines_ok

In [49]:
def generate_df(trusted_data: List[List[int]], year: str, month: str) -> pd.DataFrame:
    """
    """
    # Corrige data: 24h00 -> 00h00
    for i, day_indexex in enumerate(trusted_data):
        trusted_data[i].insert(0, day_indexex.pop())
   
    df = pd.DataFrame(trusted_data, columns=list(range(0, 24)))
    df.index = df.index+1
    df['date'] = [pd.to_datetime(f"{year}-{month}-{day}", format='%Y-%m-%d') for day in df.index]
    df['dst_min'] = [df.loc[i, np.array(range(0, 24))].min() for i in range(1, len(df)+1)]
    return df

In [50]:
def make_classification(df: pd.DataFrame, classification_rules: Dict[str, List[int]]):
    """
    df [DataFrame]
    classification_rules [Dict[str, List[int]]]
        example:
            {
                'fraca':            np.array(range(-31, -51, -1)),
                'moderada':         np.array(range(-51, -101, -1)),
                'intensa':          np.array(range(-101, -251, -1)),
                'super_intensa':    np.array(range(-251, -1001, -1)),
            }
    """
    df['classification'] = np.nan
    for i in range(1, len(df)+1):
        for category, index_range in classification_rules.items(): 
            if df.loc[i, 'dst_min'] in index_range:
                df.loc[i, 'classification'] = category
                break

In [51]:
def remove_storms_by_date(df: pd.DataFrame, dates: List[str]):
    """
    df: DataFrame
        columns: 
            date pd.datetime
    dates: list[str]
        format: YYYY-MM-DD
    """
    format_dates = [pd.to_datetime(date, format='%Y-%m-%d') for date in dates]
    boolean_mask = [date not in format_dates for date in df['date']]
    final_mask = pd.Series(boolean_mask, name='date', index=list(range(1, len(df)+1)))
    filtered_df = df[final_mask]
    filtered_df.reset_index(drop=True, inplace=True)
    return filtered_df

In [52]:
def save_processed_data(df, year, month):
    processed_data = pd.HDFStore(path=f'processed_data/{year}{month}.h5')
    processed_data.append('df', df)
    processed_data.close()

In [53]:
def plot_dst_graph(df):
    # alta resolução
    axis_x = [(i+1)/df.columns.size for i in range(df.columns.size*len(df))]

    elements = [df.iloc[i].to_list() for i in range(len(df))]
    axis_y = list()
    for element in elements:
        axis_y += element

    plt.plot(axis_x, axis_y)

    # baixa resolução
    # df.min(axis=1).plot()

In [54]:
_ = \
"""
[Fraca]          -30 nT > Dst >=  -50 nT
[Moderada]       -50 nT > Dst >= -100 nT
[Intensa]       -100 nT > Dst >= -250 nT
[SuperIntensa]  -250 nT > Dst 
"""

classification_rules = {
    'fraca':            np.array(range(-31, -51, -1)),
    'moderada':         np.array(range(-51, -101, -1)),
    'intensa':          np.array(range(-101, -251, -1)),
    'super_intensa':    np.array(range(-251, -1001, -1)),
}

In [55]:
year = '2003'
month = '11'

month_data = get_month_data(year=year, month=month)
cleaned_data = clean_month_data_text(data_text=month_data)
trusted_data = clean_multiples_lines(lines=cleaned_data)


In [56]:
df = generate_df(trusted_data=trusted_data, year=year, month=month)
make_classification(df, classification_rules)        

In [57]:
storms_to_remove_by_date = [
    '2000-07-01',
    '2000-07-02',
    '2000-07-03',
    '2000-07-29',
    '2000-07-30'
]
df_filtered = remove_storms_by_date(df, storms_to_remove_by_date).dropna()

In [58]:
df_filtered

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,date,dst_min,classification
0,-37,-69,-69,-61,-62,-61,-62,-56,-48,-47,...,-35,-31,-28,-32,-36,-33,-31,2003-11-01,-69,moderada
1,-32,-41,-40,-31,-23,-27,-30,-26,-23,-23,...,-28,-23,-23,-21,-23,-22,-26,2003-11-02,-41,fraca
2,-11,-36,-38,-32,-26,-26,-28,-21,-18,-17,...,-2,-3,-2,-4,-5,-6,-6,2003-11-03,-38,fraca
3,-42,-20,-15,-12,-12,-14,-15,14,10,6,...,-55,-50,-45,-41,-39,-36,-39,2003-11-04,-69,moderada
4,-22,-41,-36,-31,-31,-31,-30,-27,-24,-25,...,-17,-16,-17,-12,-17,-18,-17,2003-11-05,-41,fraca
8,-25,2,2,5,3,3,-1,-3,-7,-11,...,-21,-22,-26,-33,-33,-26,-24,2003-11-09,-33,fraca
9,-37,-23,-16,-13,-21,-18,-20,-20,-13,-11,...,-20,-16,-29,-36,-32,-29,-30,2003-11-10,-37,fraca
10,-46,-48,-40,-28,-30,-42,-46,-43,-39,-37,...,-52,-51,-51,-52,-49,-48,-54,2003-11-11,-62,moderada
11,-27,-47,-38,-39,-35,-35,-32,-33,-34,-39,...,-36,-29,-33,-35,-31,-30,-27,2003-11-12,-47,fraca
12,-55,-29,-29,-25,-18,-23,-31,-31,-26,-33,...,-44,-52,-46,-59,-59,-55,-47,2003-11-13,-59,moderada


In [68]:
def class_count(df: pd.DataFrame):
    count = {"fraca": 0, "moderada": 0, "intensa": 0, "super_intensa": 0}
    for i in range(1, len(df)):
        count[df['classification'].iloc[i]]+=1
    print(count)
class_count(df_filtered)

{'fraca': 13, 'moderada': 6, 'intensa': 0, 'super_intensa': 2}
